In [ ]:
from gerda.core.virusPropagationModel import *
import gerda.utilities.VPM_plotting as vpm_plot
%matplotlib inline
%load_ext autoreload
import glob
import os
import copy
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.cm as cm
#%load_ext memory_profiler

In [ ]:
import time
t0=time.time()

In [ ]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [ ]:
geopath = 'input_data/geo/'
geofiles = {0:'Buildings_Gangelt_MA_1.csv',
            1:'Buildings_Gangelt_MA_3.csv',
           }         

# Build world 

### Choose which world to initate

Here we choose, which version of the world to use in this notebook.
By default we use the reduced version of the world, in order to avoid excessive runtimes.
However one can comment out the second line and uncomment the first line, to run the screen on the complete world (Gangelt town), we based the results in the manuscript on.

Please note: <br>
The approximate runtime on our all-purpose machines amounts to 5 Minutes for the reduced Gangelt and 20 Minutes for the complete Gangelt.

In [ ]:
world_to_pick = 0 ## complete version of modelled town (Gangelt)
#world_to_pick = 1 ## reduced version of modelled town (Gangelt)

choose the number of initially infected agents

In [ ]:
n_initially_infected=4

initiate world, according to specifications

In [ ]:
modeledWorld = ModeledPopulatedWorld(initial_infections=n_initially_infected, geofile_name=geopath+geofiles[world_to_pick], input_schedules='schedules_v2')
n_people = modeledWorld.number_of_people
print('Number of agents: '+ str(n_people))

# Info on world

### Plot histogram of age-distribution (blue) and ages of initially infected agents (red)

In [ ]:
modeledWorld.plot_initial_distribution_of_ages_and_infected(age_groups_step=10)

### Get tabular information on agent-population (Agent-ID, age, home-location/household, agent-/schedule type and interactivity)

In [ ]:
df_loc = modeledWorld.get_agent_info()
df_loc.head(4)

### Get tabular information on agent-population (Location-ID, location-type, area, coordinates ...)

In [ ]:
df_ag = modeledWorld.get_location_info()
df_ag.head(4)

### Get number of locations of different types as table

In [ ]:
modeledWorld.get_distribution_of_location_types()

### Plot the distribution of different agent-/schedule types and location-types (homes are ignored, due to large number and the mixing-location is only relevant for the homogeneous-mixing version of the model)

In [ ]:
modeledWorld.plot_locations_and_schedules(locs_to_hide=['home','mixing_loc'])

# Sample simulation

### Define characteristics of the simulation to be performed

timesteps: Number of simulated timesteps in hours <br>
general_infectivity: Scaling factor for the general infectivity (default value = 0.3) <br>
general_interaction_frequency: Expected interaction-frequency over all individuals (default-value=2/h)

In [ ]:
timesteps=1000
general_infectivity=0.3
general_interaction_frequency=2

### Initiate simulation from world

Please note, that applying the copy.deepcopy function to the world, used to initiate the simulation, 
is not necessary in this context; however we include it to comply with our coding convention.

In [ ]:
simulation = Simulation(copy.deepcopy(modeledWorld),timesteps,run_immediately=False)

### Set agent infectivities to specified value and general interaction frequency to the specified value

Some description

In [ ]:
simulation.change_agent_attributes({'all':{'behaviour_as_infected':{'value':general_infectivity,'type':'replacement'}}})
simulation.interaction_frequency=general_interaction_frequency

### Run simulation with specified characteristics

In [ ]:
simulation.simulate()

## Plotting simulation outcome

### Plot trajectories of agent-states and sub-states in the whole population

In [ ]:
simulation.plot_status_timecourse()
simulation.plot_flags_timecourse(specific_flags=['Diagnosed','WasDiagnosed','Hospitalized','WasHospitalized','ICUed','WasICUed'])

### Plot state-trajectories, resolved by age-cohort

In [ ]:
simulation.plot_age_groups_status_timecourse()

### Plot heatmaps of age-specific interaction and infection-patterns

In [ ]:
simulation.plot_interaction_patterns()
simulation.plot_infection_patterns()

### Plot the relative over- and under-representation in the infection transmission of different location and agent types

In [ ]:
simulation.plot_infections_per_schedule_type_delta(modeledWorld,relative=False, label_offset=0.04, title='all infections',save_figure=False)
simulation.plot_infections_per_location_type_delta(modeledWorld,relative=False, label_offset=0.04,title='Infections per location-type',save_figure=False)

### Plot the distribution of average hourly total- and unique contacts per agent, over one week

In [ ]:
simulation.plot_contact_distributions_as_violins(ScheduleType_name_map={'under_age': 'UA', 'adult': 'AD', 'pensioner': 'PE', 'public_worker': 'PW', 'teacher': 'TE', 'medical_professional': 'MD'})

In [ ]:
print(time.time()-t0)

In [ ]:
#small: 3min